In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from astropy.modeling import models, fitting

# Configuración de estilo para los gráficos
plt.style.use('default')
sns.set_palette("colorblind")

def analyze_photometry_coherence(df, aperture=4):
    """
    Analizar la coherencia entre las magnitudes SPLUS y las del catálogo de Taylor
    """
    # Definir las correspondencias entre filtros
    filter_correspondences = [
        ('MAG_F378', 'umag', 'F378 vs u-band'),
        ('MAG_F395', 'umag', 'F395 vs u-band (aproximado)'),
        ('MAG_F410', 'gmag', 'F410 vs g-band (aproximado)'),
        ('MAG_F430', 'gmag', 'F430 vs g-band (aproximado)'),
        ('MAG_F515', 'gmag', 'F515 vs g-band'),
        ('MAG_F660', 'rmag', 'F660 vs r-band'),
        ('MAG_F861', 'imag', 'F861 vs i-band')
    ]
    
    results = {}
    n_plots = len(filter_correspondences)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.ravel()
    
    for i, (splus_filter, taylor_filter, title) in enumerate(filter_correspondences):
        splus_col = f'{splus_filter}_{aperture}'
        
        # Verificar que las columnas existen
        if splus_col not in df.columns or taylor_filter not in df.columns:
            print(f"Advertencia: {splus_col} o {taylor_filter} no encontrados. Saltando.")
            continue
            
        # Filtrar datos válidos (excluyendo valores 99.0 y NaN)
        valid_mask = (
            (df[splus_col] < 90) & 
            (df[taylor_filter] < 90) &
            (df[splus_col] > 10) &
            (df[taylor_filter] > 10) &
            (~df[splus_col].isna()) &
            (~df[taylor_filter].isna())
        )
        
        splus_mags = df.loc[valid_mask, splus_col]
        taylor_mags = df.loc[valid_mask, taylor_filter]
        
        if len(splus_mags) < 10:
            print(f"No hay suficientes datos válidos para {splus_col} vs {taylor_filter}. Saltando.")
            continue
        
        # Calcular diferencias
        differences = splus_mags - taylor_mags
        
        # Estadísticas
        mean_diff = np.mean(differences)
        median_diff = np.median(differences)
        std_diff = np.std(differences)
        mad_diff = np.median(np.abs(differences - median_diff))  # Desviación absoluta mediana
        correlation = stats.pearsonr(taylor_mags, splus_mags)[0]
        
        # Ajustar una línea recta
        try:
            init_model = models.Linear1D(slope=1, intercept=0)
            fitter = fitting.LinearLSQFitter()
            fitted_model = fitter(init_model, taylor_mags, splus_mags)
            slope, intercept = fitted_model.slope.value, fitted_model.intercept.value
        except:
            slope, intercept = 1, 0
        
        results[splus_filter] = {
            'taylor_filter': taylor_filter,
            'slope': slope,
            'intercept': intercept,
            'correlation': correlation,
            'mean_diff': mean_diff,
            'median_diff': median_diff,
            'std_diff': std_diff,
            'mad_diff': mad_diff,
            'n_sources': len(splus_mags)
        }
        
        # Graficar
        ax = axes[i]
        sc = ax.scatter(taylor_mags, splus_mags, alpha=0.6, s=15, c=differences, 
                       cmap='coolwarm', vmin=-1, vmax=1)
        
        # Línea de 1:1
        x_range = np.linspace(min(taylor_mags), max(taylor_mags), 100)
        ax.plot(x_range, x_range, 'k--', alpha=0.7, label='1:1')
        
        # Línea ajustada
        ax.plot(x_range, slope*x_range + intercept, 'r-', 
                label=f'y = {slope:.3f}x + {intercept:.3f}')
        
        ax.set_xlabel(f'Taylor {taylor_filter}')
        ax.set_ylabel(f'SPLUS {splus_filter}')
        ax.set_title(f'{title}\nr = {correlation:.3f}, Δ = {median_diff:.3f} ± {mad_diff:.3f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Añadir barra de color para las diferencias
        plt.colorbar(sc, ax=ax, label='Diferencia (SPLUS - Taylor)')
    
    # Ocultar ejes vacíos
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('../anac_data/splus_taylor_coherence_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return results

def generate_statistical_summary(results):
    """
    Generar un resumen estadístico de la comparación
    """
    print("="*60)
    print("ANÁLISIS DE COHERENCIA: FOTOMETRÍA SPLUS vs TAYLOR")
    print("="*60)
    
    summary_data = []
    for splus_filter, stats in results.items():
        summary_data.append({
            'Filtro SPLUS': splus_filter,
            'Filtro Taylor': stats['taylor_filter'],
            'N': stats['n_sources'],
            'Correlación': f"{stats['correlation']:.3f}",
            'Pendiente': f"{stats['slope']:.3f}",
            'Intercepto': f"{stats['intercept']:.3f}",
            'Δ media': f"{stats['mean_diff']:.3f}",
            'Δ mediana': f"{stats['median_diff']:.3f}",
            'σ': f"{stats['std_diff']:.3f}",
            'MAD': f"{stats['mad_diff']:.3f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    # Guardar resumen en CSV
    summary_df.to_csv('../anac_data/splus_taylor_coherence_summary.csv', index=False)
    print(f"\nResumen guardado en 'splus_taylor_coherence_summary.csv'")

# Cargar los datos
df = pd.read_csv('../anac_data/all_fields_gc_photometry_merged.csv')

# Analizar coherencia (usando apertura de 4 arcsec por defecto)
results = analyze_photometry_coherence(df, aperture=4)

# Generar resumen estadístico
generate_statistical_summary(results)

# Análisis adicional: Distribución de diferencias por filtro
plt.figure(figsize=(12, 8))
for splus_filter, stats in results.items():
    # Filtrar datos válidos
    valid_mask = (
        (df[f'{splus_filter}_4'] < 90) & 
        (df[stats['taylor_filter']] < 90) &
        (df[f'{splus_filter}_4'] > 10) &
        (df[stats['taylor_filter']] > 10) &
        (~df[f'{splus_filter}_4'].isna()) &
        (~df[stats['taylor_filter']].isna())
    )
    
    differences = df.loc[valid_mask, f'{splus_filter}_4'] - df.loc[valid_mask, stats['taylor_filter']]
    
    # Graficar histograma de diferencias
    plt.hist(differences, bins=30, alpha=0.5, 
             label=f'{splus_filter} - {stats["taylor_filter"]} (μ={np.mean(differences):.3f})')

plt.xlabel('Diferencia (SPLUS - Taylor)')
plt.ylabel('Frecuencia')
plt.title('Distribución de diferencias entre fotometría SPLUS y Taylor')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('../anac_data/splus_taylor_differences_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

Advertencia: MAG_F378_4 o umag no encontrados. Saltando.
Advertencia: MAG_F395_4 o umag no encontrados. Saltando.
Advertencia: MAG_F410_4 o gmag no encontrados. Saltando.
Advertencia: MAG_F430_4 o gmag no encontrados. Saltando.
Advertencia: MAG_F515_4 o gmag no encontrados. Saltando.
Advertencia: MAG_F660_4 o rmag no encontrados. Saltando.
Advertencia: MAG_F861_4 o imag no encontrados. Saltando.


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


ANÁLISIS DE COHERENCIA: FOTOMETRÍA SPLUS vs TAYLOR
Empty DataFrame
Columns: []
Index: []

Resumen guardado en 'splus_taylor_coherence_summary.csv'
